In [ ]:
!pip install opencv-python==4.8.0.74
!pip install timm
!pip install torcheval

In [ ]:
import torch
import random
import numpy as np

def set_seed(seed):
    random.seed(seed)                           # Python의 random 시드 설정
    np.random.seed(seed)                        # Numpy의 random 시드 설정
    torch.manual_seed(seed)                     # PyTorch CPU 시드 설정
    torch.cuda.manual_seed(seed)                # PyTorch GPU 시드 설정
    torch.cuda.manual_seed_all(seed)            # 모든 GPU의 시드를 설정 (멀티 GPU 환경)
    
    # CuDNN의 비결정적 연산을 방지하여 재현성을 높입니다.
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)
random_state = 42

# GPU 사용 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import os
import gc
import cv2
import math
import copy
import time
import random
import glob
from matplotlib import pyplot as plt
from PIL import Image
from io import BytesIO

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
import torchvision

# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict

# For Image Models
import timm

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from glob import glob
from sklearn.model_selection import train_test_split

# 데이터셋 경로 설정
base_dir = './data/Training'
noisy_image_paths = os.path.join(base_dir, 'noisy')

# 각 유형별로 리스트 초기화
ff_rf_paths = []
rl_ud_paths = []
sc_le_paths = []
other_blur_paths = []

# 이미지 파일 리스트 생성
all_image_paths = glob(os.path.join(noisy_image_paths, '*.jpg'))

# 파일 경로 리스트 분류
for image_path in all_image_paths:
    filename = os.path.basename(image_path)
    blur_type = filename.split('_')[-1][:2]  # 블러 유형 추출 (FI 등)

    if blur_type in ['FF', 'RF']:
        ff_rf_paths.append(image_path)
    elif blur_type in ['RL', 'UD']:
        rl_ud_paths.append(image_path)
    elif blur_type in ['SC', 'LE']:
        sc_le_paths.append(image_path)
    else:
        other_blur_paths.append(image_path)

# 경로와 레이블 생성
data_paths = ff_rf_paths + rl_ud_paths + sc_le_paths + other_blur_paths
labels = [0] * len(ff_rf_paths) + [1] * len(rl_ud_paths) + [2] * len(sc_le_paths) + [3] * len(other_blur_paths)

# Train/Test Split
train_paths, val_paths, train_labels, val_labels = train_test_split(data_paths, labels, test_size=0.05, random_state=42)

# 데이터셋 클래스 정의
class BlurDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# 이미지 변환 설정
train_transform = transforms.Compose([
    transforms.CenterCrop(1080),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.CenterCrop(1080),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 데이터 로더 생성
train_dataset = BlurDataset(train_paths, train_labels, transform=train_transform)
val_dataset = BlurDataset(val_paths, val_labels, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2)

In [ ]:
# ResNet 모델 정의 및 수정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 4)  # 4가지 블러 유형으로 분류하도록 출력층 수정
model = model.to(device)

# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)
epochs_no_improve = 0
best_loss = float('inf')
best_model = None
# 학습 및 평가 루프
num_epochs = 20
for epoch in range(num_epochs):
    # Training
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels).item()
    val_loss /= len(val_loader.dataset)
    val_acc = correct / len(val_loader.dataset)

    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_loss:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

    # Check for improvement
    if val_loss < best_loss:
        best_loss = val_loss
        best_model = model
        epochs_no_improve = 0  # 개선되었으면 카운트 초기화
    else:
        epochs_no_improve += 1

    # Early stopping check
    if epochs_no_improve >= 4:
        print(f'Early stopping at epoch {epoch} due to no improvement in validation loss.')
        break
print("Training complete.")

In [ ]:
torch.save(best_model.state_dict(), 'resnet18_0.9621_90퍼.pth')